<a href="https://colab.research.google.com/github/raminbazr/Python/blob/main/Multi-Variate-LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Multi-Variate LSTM**

In [ ]:
from sklearn.metrics import mean_squared_error
import tensorflow as tf
from tensorflow.keras.layers import Embedding, Dense, LSTM
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow import keras
from matplotlib import pyplot
from keras.layers import Dropout
from keras import regularizers
from keras.callbacks import EarlyStopping
import math,time
from sklearn.preprocessing import StandardScaler

#df = df.drop(["Torque"], axis =1)
#normalize the dataset
scaler = StandardScaler()
df = scaler.fit_transform(df)


n_future =1
look_back=1
trainX = []
trainY = []
# convert an array of values into a dataset matrix
for i in range(look_back,len(df)-n_future +1):
	trainX.append(df[i-look_back: i,0:df.shape[1]])
	trainY.append(df[i+n_future-1:i+n_future,4])

trainX , trainY= np.array(trainX),np.array(trainY)
 
# create and fit the LSTM network
modellstmROP = Sequential()
modellstmROP.add(LSTM(11,return_sequences=True, input_shape=(trainX.shape[1],trainX.shape[2])))
modellstmROP.add(Dropout(0.17))
modellstmROP.add(LSTM(units=11, return_sequences=False))
modellstmROP.add(Dropout(0.17))
modellstmROP.add(Dense(1))

lr_schedule = tf.keras.callbacks.LearningRateScheduler(
    lambda epoch: 1e-8 * 10**(epoch / 20))
optimizer = tf.keras.optimizers.SGD(learning_rate=1e-8, momentum=0.9)

modellstmROP.compile(loss='mean_squared_error', optimizer='adam')
history=modellstmROP.fit(trainX, trainY, epochs=100, batch_size=4, verbose=2,callbacks=[lr_schedule])

In [ ]:
print('trainX shape == {}.'.format(trainX.shape))
print('trainY shape == {}.'.format(trainY.shape))

In [ ]:
plt.semilogx(history.history["lr"], history.history["loss"])

In [ ]:
train = np.array (train)
test = np.array (test)

In [ ]:
# by above plot which is for learning rate , we get learning rate= 1e-4 because in this rate we have less changing
from sklearn.metrics import mean_squared_error
import tensorflow as tf
from tensorflow.keras.layers import Embedding, Dense, LSTM
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow import keras
from matplotlib import pyplot
from keras.layers import Dropout
from keras import regularizers
from keras.callbacks import EarlyStopping
import math,time
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.callbacks import ModelCheckpoint


n_future =1
look_back=1
trainX = []
trainY = []
# convert an array of values into a dataset matrix
for i in range(look_back,len(train)-n_future +1):
	trainX.append(train[i-look_back: i,0:train.shape[0]])
	trainY.append(train[i+n_future-1:i+n_future,4])

trainX , trainY= np.array(trainX),np.array(trainY)

# create and fit the LSTM network
modellstmROP = Sequential()
modellstmROP.add(LSTM(11,return_sequences=True, input_shape=(1,15)))
modellstmROP.add(Dropout(0.17))
modellstmROP.add(LSTM(units=11, return_sequences=False))
modellstmROP.add(Dropout(0.17))
modellstmROP.add(Dense(1))


modellstmROP.compile(loss='mean_squared_error', optimizer=tf.keras.optimizers.SGD(learning_rate=1e-3, momentum=0.9),metrics=["mae"])

history=modellstmROP.fit(trainX, trainY, epochs=100, batch_size=4, validation_split = 0.1,verbose=0)

In [ ]:
# convert an array of values into a dataset matrix
testX = []
testY = []
for i in range(look_back,len(test)-n_future +1):
	testX.append(test[i-look_back: i,0:test.shape[1]])
	testY.append(test[i+n_future-1:i+n_future,3])
 
testX , testY= np.array(testX),np.array(testY)

In [ ]:
print(trainX.shape)
print(testX.shape)
print(trainY.shape)
print(testY.shape)

In [ ]:
print('ROP STD :',np.std(df[4]))
print('ROP mean:',np.mean(df[4]))
print('Torque STD :',np.std(df[1]))
print('Torque mean:',np.mean(df[1]))

In [ ]:
import matplotlib.image  as mpimg
import matplotlib.pyplot as plt

#-----------------------------------------------------------
# Retrieve a list of list results on training and test data
# sets for each training epoch
#-----------------------------------------------------------
mae=history.history['mae']
loss=history.history['loss']

epochs=range(len(loss)) # Get number of epochs

#------------------------------------------------
# Plot MAE and Loss
#------------------------------------------------
plt.plot(epochs, mae, 'r')
plt.plot(epochs, loss, 'b')
plt.title('MAE and Loss')
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend(["MAE", "Loss"])

plt.figure()

epochs_zoom = epochs[50:]
mae_zoom = mae[50:]
loss_zoom = loss[50:]

#------------------------------------------------
# Plot Zoomed MAE and Loss
#------------------------------------------------
plt.plot(epochs_zoom, mae_zoom, 'r')
plt.plot(epochs_zoom, loss_zoom, 'b')
plt.title('MAE and Loss')
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend(["MAE", "Loss"])

plt.figure()

In [ ]:
# make predictions
trainPredict = modellstmROP.predict(trainX)
testPredict = modellstmROP.predict(testX)

# calculate root mean squared error
trainScore = math.sqrt(mean_squared_error(trainY, trainPredict))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(testY, testPredict))
print('Test Score: %.2f RMSE' % (testScore))


# shift train predictions for plotting
trainPredictPlot = np.repeat(trainPredict,14, axis=-1)
testPredictPlot=np.repeat(testPredict,14, axis=-1)
# plot baseline and predictions
#plt.plot(scaler.inverse_transform(df))
plt.plot(trainPredictPlot)
plt.plot(testPredictPlot)
plt.show()

In [ ]:
plt.plot(trainY, color ='b')
plt.plot(trainPredict, color = 'r')

In [ ]:
plt.plot(testY, color ='b')
plt.plot(testPredict, color = 'r')

In [ ]:
(testPredict*np.std(df[4]))

In [ ]:
from sklearn.metrics import mean_squared_error
rmse = np.sqrt(mean_squared_error(testY, testPredict))
print('RMSE: %.3f' % rmse)

In [ ]:
NRMSE = (rmse/(y_test.max()-y_test.min()))*100
NRMSE

In [ ]:
import tensorflow
tensorflow.keras.metrics.mean_absolute_error(testY, testPredict)

In [ ]:
from sklearn.metrics import mean_absolute_percentage_error
mean_absolute_percentage_error(testY, testPredict)*100

In [ ]:
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(testY, testPredict)
print(mae)

In [ ]:
from sklearn.metrics import r2_score

r2_score(testY, testPredict)

In [ ]:
x_ax = range(len(testPredict))
plt.scatter(x_ax, testY, s=5, color="blue", label="original")
plt.plot(x_ax, testPredict, lw=0.8, color="red", label="predicted")
plt.legend()
plt.show()